### Importing libraries...

In [1]:
import requests
from pymongo import MongoClient
import pandas as pd
import re
import json
from pandas.io.json import json_normalize
import os
import numpy as np

### Importing Companies Data

In [2]:
dbName = "companies.json"
mongodbURL = f"mongodb://localhost/{dbName}"

client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

### Verifying amount of companies of potential chosen country

In [3]:
data = db.companies.aggregate([{"$group":{"_id":"$offices.country_code","count":{"$sum":1}}}])
countries = list(data)

In [4]:
countries

[{'_id': ['BRA'], 'count': 16},
 {'_id': ['ESP', 'CHL'], 'count': 1},
 {'_id': ['DEU', 'USA', 'JPN', 'CHN'], 'count': 1},
 {'_id': ['USA', 'SWE', 'MYS', 'JPN', 'CHN'], 'count': 2},
 {'_id': ['USA', 'DEU', 'DEU'], 'count': 1},
 {'_id': ['ESP', 'GBR'], 'count': 1},
 {'_id': ['USA', 'GBR', 'THA'], 'count': 1},
 {'_id': ['USA', 'USA', 'GBR', 'USA'], 'count': 2},
 {'_id': ['AUS', 'ISR'], 'count': 1},
 {'_id': ['ESP', 'GBR', 'GBR', 'GBR'], 'count': 1},
 {'_id': ['IND', 'SWE'], 'count': 1},
 {'_id': ['GBR', 'USA', 'USA', 'SGP'], 'count': 1},
 {'_id': ['USA', 'ITA'], 'count': 3},
 {'_id': ['ESP', 'FRA'], 'count': 1},
 {'_id': ['USA', 'USA', 'IND', 'GBR'], 'count': 1},
 {'_id': ['USA', 'USA', 'GBR', 'AUS'], 'count': 1},
 {'_id': ['NGA'], 'count': 2},
 {'_id': ['SVN'], 'count': 7},
 {'_id': ['USA', 'GBR', 'USA', 'JPN', 'USA'], 'count': 1},
 {'_id': ['USA', 'USA', 'USA', 'GBR'], 'count': 6},
 {'_id': ['FRA', 'GBR', 'USA', 'GBR'], 'count': 2},
 {'_id': ['FRA', 'USA', 'GBR', 'JPN'], 'count': 1},
 {

In [5]:
county_counts=[]
for x in countries:
    county_counts.append(x['_id'])


In [6]:
county_counts = [c for n in county_counts for c in n]
county_counts

['BRA',
 'ESP',
 'CHL',
 'DEU',
 'USA',
 'JPN',
 'CHN',
 'USA',
 'SWE',
 'MYS',
 'JPN',
 'CHN',
 'USA',
 'DEU',
 'DEU',
 'ESP',
 'GBR',
 'USA',
 'GBR',
 'THA',
 'USA',
 'USA',
 'GBR',
 'USA',
 'AUS',
 'ISR',
 'ESP',
 'GBR',
 'GBR',
 'GBR',
 'IND',
 'SWE',
 'GBR',
 'USA',
 'USA',
 'SGP',
 'USA',
 'ITA',
 'ESP',
 'FRA',
 'USA',
 'USA',
 'IND',
 'GBR',
 'USA',
 'USA',
 'GBR',
 'AUS',
 'NGA',
 'SVN',
 'USA',
 'GBR',
 'USA',
 'JPN',
 'USA',
 'USA',
 'USA',
 'USA',
 'GBR',
 'FRA',
 'GBR',
 'USA',
 'GBR',
 'FRA',
 'USA',
 'GBR',
 'JPN',
 'USA',
 'USA',
 'CHN',
 'CHN',
 'IRQ',
 'USA',
 'USA',
 'USA',
 'AUS',
 'GBR',
 'FIN',
 'CAN',
 'USA',
 'FRA',
 'FRA',
 'DEU',
 'DEU',
 'DEU',
 'FRA',
 'NLD',
 'ESP',
 'GBR',
 'USA',
 'PRT',
 'KEN',
 'NLD',
 'NLD',
 'BEL',
 'FRA',
 'USA',
 'USA',
 'EST',
 'USA',
 'IRL',
 'USA',
 'DNK',
 'USA',
 'IDN',
 'PHL',
 'SWE',
 'SWE',
 'NLD',
 'FIN',
 'USA',
 'USA',
 'GBR',
 'JPN',
 'AUS',
 'ARG',
 'BRA',
 'MEX',
 'ESP',
 'FRA',
 'GRC',
 'DEU',
 'CZE',
 'POL',
 'RUS',


In [7]:
from collections import Counter
county_counts = Counter(county_counts)
{k: v for k, v in sorted(county_counts.items(), key=lambda item: item[1], reverse=True)}

{'USA': 775,
 'GBR': 257,
 'DEU': 116,
 'IND': 106,
 'FRA': 82,
 'AUS': 80,
 'CHN': 79,
 'ESP': 62,
 'CAN': 62,
 'JPN': 52,
 'SGP': 46,
 'NLD': 40,
 'SWE': 35,
 'CHE': 31,
 'BRA': 30,
 'ARG': 29,
 'ISR': 28,
 'ITA': 28,
 'IRL': 25,
 'HKG': 25,
 'RUS': 23,
 'TWN': 20,
 'BEL': 19,
 'MEX': 18,
 'NOR': 17,
 'DNK': 16,
 'PHL': 15,
 'ROM': 15,
 'ZAF': 15,
 'KOR': 15,
 'NZL': 15,
 'MYS': 14,
 'ARE': 14,
 'AUT': 14,
 'POL': 13,
 'FIN': 12,
 'PRT': 12,
 'UKR': 12,
 'CZE': 11,
 'TUR': 11,
 'CHL': 9,
 'VNM': 9,
 'BGR': 8,
 'GRC': 7,
 'THA': 6,
 'SVN': 6,
 'COL': 6,
 'EST': 5,
 'EGY': 5,
 'HUN': 5,
 'ANT': 5,
 'CYP': 4,
 'LVA': 4,
 'HRV': 4,
 'PAK': 4,
 'LKA': 4,
 'NGA': 3,
 'IDN': 3,
 'LTU': 3,
 'MLT': 3,
 'JOR': 3,
 'SAU': 3,
 'MKD': 3,
 'LUX': 3,
 'BGD': 3,
 'URY': 3,
 'MAR': 3,
 'KEN': 2,
 'LBN': 2,
 'VGB': 2,
 'BIH': 2,
 'ARA': 2,
 'GIB': 2,
 'QAT': 2,
 'UZB': 2,
 'PAN': 2,
 'NIC': 2,
 'ISL': 2,
 'MUS': 2,
 'VEN': 2,
 'CSS': 2,
 'PRK': 2,
 'IRQ': 1,
 'PST': 1,
 'BFA': 1,
 'REU': 1,
 'IOT': 1,

**Countries which companies I have more information of:**  
  
USA 🇺🇸  
GBR 🇬🇧  
DEU 🇩🇪   
IND 🇮🇪  
FRA 🇫🇷  

**Top Videogame development Countries:**

USA 🇺🇸 = 194  
GBR 🇬🇧 =  48  
DEU 🇩🇪 =   /  
IND 🇮🇪  =   /  
FRA 🇫🇷 =  23  

**Chosen Country:  
US** 🇺🇸 🇺🇸 🇺🇸 🇺🇸

## Location

**Requisite:**  
Developers like to be near successful tech startups that have raised at least 1 Million dollars.

**Query:**  
 - Country: USA  
 - Is a startup: founded after 2007 
 - Tech categories: software, design, games_video, web, mobile
 - Raised at least 1 Million dollars 
 

In [8]:
query = {"$and":[
    {"offices.country_code":"USA"},
    {"founded_year":{"$gte":2007}},
    {"funding_rounds.raised_amount":{"$gte":1000000}},
    {"category_code":{"$in":["software","design","games_video","web", "mobile"]}},
]}
cur = db.companies.find(query)
df = pd.DataFrame(list(cur))
df

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7c4bab8bd675297da9,Mahalo,mahalo,http://www.crunchbase.com/company/mahalo,http://mahalo.com,http://blog.mahalo.com/,http://blog.mahalo.com/feed/,MahaloDotCom,web,40.0,...,[],None,[],"[{'description': '', 'address1': '3525 Eastham...",[],None,"[{'embed_code': '<embed src=""http://blip.tv/pl...",[],[],[]
1,52cdef7c4bab8bd675297dc2,AdaptiveBlue,adaptiveblue,http://www.crunchbase.com/company/adaptiveblue,http://www.getglue.com,http://blog.adaptiveblue.com,http://blog.adaptiveblue.com/?feed=rss2,GetGlue,games_video,15.0,...,[],None,[],"[{'description': '', 'address1': '131 Varick S...",[],None,"[{'embed_code': '<object width=""425"" height=""3...",[],[{'external_url': 'http://web20show.com/2009/1...,[]
2,52cdef7c4bab8bd675297dc0,Livestream,livestream,http://www.crunchbase.com/company/livestream,http://www.livestream.com,http://www.livestream.com/blog/,http://www.livestream.com/blog/?feed=rss2,livestream,games_video,120.0,...,[],"{'price_amount': None, 'price_currency_code': ...",[],"[{'description': 'Livestream HQ', 'address1': ...",[],None,"[{'embed_code': '<embed src=""http://blip.tv/pl...",[],[{'external_url': 'http://scobleizer.com/2007/...,[]
3,52cdef7c4bab8bd675297dc1,Ustream,ustream,http://www.crunchbase.com/company/ustream,http://www.ustream.tv,http://www.ustream.com/blog,http://www.ustream.com/blog/feed,USTREAM,games_video,250.0,...,[],None,[],"[{'description': 'San Francisco Office', 'addr...","[{'id': 6587, 'description': 'Ustream recogniz...",None,"[{'embed_code': '<object id=""flashObj"" width=""...","[{'available_sizes': [[[150, 103], 'assets/ima...",[{'external_url': 'http://facebook.com/ustream...,[]
4,52cdef7c4bab8bd675297dd1,SodaHead,sodahead,http://www.crunchbase.com/company/sodahead,http://sodahead.com,,,SodaHead,web,25.0,...,[],None,[],"[{'description': '', 'address1': '16161 Ventur...",[],None,[],"[{'available_sizes': [[[150, 131], 'assets/ima...",[{'external_url': 'http://www.prweb.com/releas...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,52cdef7f4bab8bd67529c4f0,Culture Jam,culture-jam,http://www.crunchbase.com/company/culture-jam,http://site.promojam.com,http://promojam.tumblr.com/,,culturejam,web,15.0,...,[],None,[],"[{'description': '', 'address1': '2434 Lincoln...",[],None,[],[],[{'external_url': 'http://www.techcrunch.com/2...,[]
295,52cdef7f4bab8bd67529c55b,Clavis Technology,clavis-technology,http://www.crunchbase.com/company/clavis-techn...,http://www.clavistechnology.com,http://www.clavistechnology.com/news,http://www.clavistechnology.com/feed/,Clavistech,software,50.0,...,[],None,[],"[{'description': '', 'address1': '7th Floor,',...","[{'id': 7702, 'description': 'Clavis Secures S...",None,"[{'embed_code': '<p> <object classid=""clsid:d...","[{'available_sizes': [[[150, 92], 'assets/imag...",[{'external_url': 'http://the451group.com/repo...,[]
296,52cdef7f4bab8bd67529c59b,Luma International,luma-international,http://www.crunchbase.com/company/luma-interna...,http://www.lumacentral.com,http://blog.lumacentral.com,http://blog.lumacentral.com/atom.xml,lumacentral,software,40.0,...,[],None,[],"[{'description': 'Headquarters', 'address1': '...",[],None,[],"[{'available_sizes': [[[150, 114], 'assets/ima...",[],[]
297,52cdef7f4bab8bd67529c5b8,Continuity Engine,continuity-engine,http://www.crunchbase.com/company/continuity-e...,http://www.continuity.net,http://continuityengine.blogspot.com/,,ContinuityEng,software,11.0,...,[],None,"[{'price_amount': None, 'price_currency_code':...","[{'description': 'HQ', 'address1': '150 Munson...",[],None,[],"[{'available_sizes': [[[150, 72], 'assets/imag...",[],[]


In [9]:
df.columns

Index(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url',
       'blog_url', 'blog_feed_url', 'twitter_username', 'category_code',
       'number_of_employees', 'founded_year', 'founded_month', 'founded_day',
       'deadpooled_year', 'deadpooled_month', 'deadpooled_day',
       'deadpooled_url', 'tag_list', 'alias_list', 'email_address',
       'phone_number', 'description', 'created_at', 'updated_at', 'overview',
       'image', 'products', 'relationships', 'competitions', 'providerships',
       'total_money_raised', 'funding_rounds', 'investments', 'acquisition',
       'acquisitions', 'offices', 'milestones', 'ipo', 'video_embeds',
       'screenshots', 'external_links', 'partners'],
      dtype='object')

In [10]:
offices = df[["name", "offices"]].explode("offices")
offices

,name,offices
0,Mahalo,"{'description': '', 'address1': '3525 Eastham ..."
1,AdaptiveBlue,"{'description': '', 'address1': '131 Varick St..."
2,Livestream,"{'description': 'Livestream HQ', 'address1': '..."
3,Ustream,"{'description': 'San Francisco Office', 'addre..."
3,Ustream,"{'description': 'Los Angeles Office', 'address..."
...,...,...
296,Luma International,"{'description': 'HQ North America', 'address1'..."
296,Luma International,"{'description': 'Office France', 'address1': '..."
296,Luma International,"{'description': 'Office The Netherlands', 'add..."
297,Continuity Engine,"{'description': 'HQ', 'address1': '150 Munson ..."


In [11]:
offices = json_normalize(json.loads(offices.to_json(orient="records")))
offices = offices[["name", "offices.latitude", "offices.longitude", "offices.state_code"]]
offices

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,name,offices.latitude,offices.longitude,offices.state_code
0,Mahalo,34.017606,-118.487267,CA
1,AdaptiveBlue,40.801358,-74.337200,NY
2,Livestream,40.726155,-73.995625,NY
3,Ustream,37.392936,-122.079480,CA
4,Ustream,NaN,NaN,CA
...,...,...,...,...
388,Luma International,NaN,NaN,CA
389,Luma International,NaN,NaN,None
390,Luma International,NaN,NaN,None
391,Continuity Engine,41.327918,-73.243552,CT


In [12]:
offices["offices.state_code"].value_counts()

CA    184
NY     50
MA     23
WA     11
TX      9
CO      9
VA      7
FL      5
IL      5
NC      5
PA      4
OR      3
GA      3
CT      3
UT      3
AZ      2
NJ      2
MD      2
MN      2
RI      1
SC      1
NH      1
DC      1
ME      1
KY      1
LA      1
Name: offices.state_code, dtype: int64

### Chosen state: California

In [13]:
offices = offices.loc[offices["offices.state_code"] == "CA"]
offices = offices.dropna()
offices = offices.reset_index()
offices = offices.drop(columns = ["index"])

In [14]:
offices

,name,offices.latitude,offices.longitude,offices.state_code
0,Mahalo,34.017606,-118.487267,CA
1,Ustream,37.392936,-122.079480,CA
2,SodaHead,37.269175,-119.306607,CA
3,Funny Or Die,37.421621,-122.137444,CA
4,Seesmic,37.775196,-122.419204,CA
...,...,...,...,...
138,Global Fitness Media,34.054318,-118.462876,CA
139,CrowdGather,34.168955,-118.575293,CA
140,High Gear Media,37.445335,-122.158330,CA
141,Teambox,41.406335,2.192023,CA


In [15]:
offices = offices.rename(columns={"name": "Name", "offices.latitude": "Latitude", "offices.longitude": "Longitude", "offices.state_code": "State Code"})

In [16]:
offices

,Name,Latitude,Longitude,State Code
0,Mahalo,34.017606,-118.487267,CA
1,Ustream,37.392936,-122.079480,CA
2,SodaHead,37.269175,-119.306607,CA
3,Funny Or Die,37.421621,-122.137444,CA
4,Seesmic,37.775196,-122.419204,CA
...,...,...,...,...
138,Global Fitness Media,34.054318,-118.462876,CA
139,CrowdGather,34.168955,-118.575293,CA
140,High Gear Media,37.445335,-122.158330,CA
141,Teambox,41.406335,2.192023,CA


## Google API

In [17]:
from dotenv import load_dotenv
load_dotenv()
apiKey = os.getenv("google_key")
url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query=starbucks+in+CA&key={apiKey}"
res = requests.get(url)


In [18]:
res = res.json()

In [19]:
from dotenv import load_dotenv
load_dotenv()
apiKey = os.getenv("google_key")
url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
#query=starbucks+in+CA&key={apiKey}&pageToken=res['next_page_token']"
res = requests.get(url)
res = res.json()

In [20]:
def getData(query, page_token=None):
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {"key":os.getenv("google_key"), "query":query}
    if page_token:
        params["pageToken"]=page_token
    res = requests.get(url, params=params)
    res = res.json()
    results = res["results"]
    if 'next_page_token' in res.keys():
        page_token = res['next_page_token']
        results=[results, page_token]
    else: 
        results=[results,False]
    return results

In [21]:
def getLocations(query, limit=50):
    data = []
    page_token = None
    while len(data)<limit:
        results, page_token = getData(query, page_token)
        data+=results
        if not page_token: 
            break
    return data

##  Función para calcular distancias

In [22]:
def FindDistances(req):
    d=[]
    for i in offices.index:
        d_list=[]
        for ind in req.index: 
            x = req['Latitude'][ind]
            y = req['Longitude'][ind] 
            distance = (((x-(offices['Latitude'][i]))**2)+((y-(offices['Longitude'][i]))**2))**(1/2)
            d_list.append(distance)
        d.append(min(d_list))
    offices['New'] = d

## Starbucks

**Requisite:**  
Executives like Starbucks A LOT. Ensure there's a starbucks not to far.

In [23]:
data = getLocations("starbucks in CA", 100)
dataset = pd.DataFrame(data)

In [24]:
starbucks = []
for st in dataset["geometry"]:
    new_dict={}
    new_dict["Latitude"]= st['location']['lat']
    new_dict["Longitude"]= st['location']['lng']
    starbucks.append(new_dict)

In [25]:
starbucks = pd.DataFrame(starbucks)
starbucks

,Latitude,Longitude
0,34.072428,-117.881891
1,37.291159,-121.989834
2,37.762265,-121.433395
3,38.714409,-120.839673
4,39.403609,-123.349905
...,...,...
95,34.136023,-118.132490
96,37.099098,-121.015467
97,38.013442,-121.868315
98,33.649162,-117.830863


In [26]:
def transformToGeoPoint(s):
    if np.isnan(s.Latitude) or np.isnan(s.Longitude):
        return None
    return {
        "type":"Point",
        "coordinates":[s.Longitude, s.Latitude]
    }
starbucks["geopoint"] = starbucks.apply(transformToGeoPoint, axis=1)
starbucks

,Latitude,Longitude,geopoint
0,34.072428,-117.881891,"{'type': 'Point', 'coordinates': [-117.881891,..."
1,37.291159,-121.989834,"{'type': 'Point', 'coordinates': [-121.9898337..."
2,37.762265,-121.433395,"{'type': 'Point', 'coordinates': [-121.4333954..."
3,38.714409,-120.839673,"{'type': 'Point', 'coordinates': [-120.839673,..."
4,39.403609,-123.349905,"{'type': 'Point', 'coordinates': [-123.349905,..."
...,...,...,...
95,34.136023,-118.132490,"{'type': 'Point', 'coordinates': [-118.13249, ..."
96,37.099098,-121.015467,"{'type': 'Point', 'coordinates': [-121.015467,..."
97,38.013442,-121.868315,"{'type': 'Point', 'coordinates': [-121.868315,..."
98,33.649162,-117.830863,"{'type': 'Point', 'coordinates': [-117.8308631..."


In [27]:
FindDistances(starbucks)

In [28]:
offices = offices.rename(columns={"New":"Starbucks"})

## Bars

**Requisite:**  
All people in the company have between 25 and 40 years, give them some place to go to party.

In [29]:
data = getLocations("School in California", 100)
barset = pd.DataFrame(data)

In [30]:
barset

,business_status,formatted_address,geometry,icon,id,name,opening_hours,photos,place_id,plus_code,rating,reference,types,user_ratings_total,permanently_closed
0,OPERATIONAL,"Travesía Navaluenga, 3, 28221 Majadahonda, Mad...","{'location': {'lat': 40.47658759999999, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,c219e5315f83966283b43ebaff32499d6f01c05a,Dreaming California. Cursos de Idiomas en el E...,{'open_now': True},"[{'height': 809, 'html_attributions': ['<a hre...",ChIJrTSVDm-EQQ0RyYTAdpxrs1E,"{'compound_code': 'F4GW+JH Majadahonda', 'glob...",4.9,ChIJrTSVDm-EQQ0RyYTAdpxrs1E,"[school, point_of_interest, establishment]",49,NaN
1,OPERATIONAL,"16800 Shoemaker Ave, Cerritos, CA 90703, Unite...","{'location': {'lat': 33.8780995, 'lng': -118.0...",https://maps.gstatic.com/mapfiles/place_api/ic...,e44a99e6ee02df0ccf3e1fa378c8c8874227828a,Gretchen Whitney High School,{'open_now': False},"[{'height': 1836, 'html_attributions': ['<a hr...",ChIJm5v7uvos3YAR9BynbzEjPTs,"{'compound_code': 'VWHW+67 Cerritos, Californi...",3.7,ChIJm5v7uvos3YAR9BynbzEjPTs,"[secondary_school, school, point_of_interest, ...",22,NaN
2,OPERATIONAL,"1000 E Victoria St, Carson, CA 90747, United S...","{'location': {'lat': 33.8610309, 'lng': -118.2...",https://maps.gstatic.com/mapfiles/place_api/ic...,f138bf0988006c70817dd9f5c04a27112abd89e0,California Academy of Mathematics and Science ...,NaN,"[{'height': 2953, 'html_attributions': ['<a hr...",ChIJv44FUNbKwoARj_BN6LF-lYU,"{'compound_code': 'VP6W+C3 Carson, California,...",4.4,ChIJv44FUNbKwoARj_BN6LF-lYU,"[secondary_school, school, point_of_interest, ...",51,NaN
3,OPERATIONAL,"3232 California St, Costa Mesa, CA 92626, Unit...","{'location': {'lat': 33.6888923, 'lng': -117.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,f3e6748380006d37f808824653cac84b1abe1217,California Elementary School,{'open_now': False},"[{'height': 1562, 'html_attributions': ['<a hr...",ChIJba0i_Kkg3YARoI91htg7ER8,"{'compound_code': 'M3QF+HQ Costa Mesa, Califor...",4.3,ChIJba0i_Kkg3YARoI91htg7ER8,"[primary_school, school, point_of_interest, es...",7,NaN
4,OPERATIONAL,"2611 E. Matoian Way, MS/UH134, Fresno, CA 9374...","{'location': {'lat': 36.810279, 'lng': -119.74...",https://maps.gstatic.com/mapfiles/place_api/ic...,10883cba7608c8351b7628ae9c38530db9acbf71,University High School,NaN,"[{'height': 1836, 'html_attributions': ['<a hr...",ChIJOQvpSAFdlIAR1zcivLGQJd0,"{'compound_code': 'R764+47 Fresno, California,...",4.4,ChIJOQvpSAFdlIAR1zcivLGQJd0,"[school, point_of_interest, establishment]",21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CLOSED_TEMPORARILY,"302 Portola Rd, Portola Valley, CA 94028, Unit...","{'location': {'lat': 37.375907, 'lng': -122.21...",https://maps.gstatic.com/mapfiles/place_api/ic...,7aec589fb8878e9aea5e31b9e3e937348f4de862,Woodside Priory School,NaN,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJbQOGXoevj4ARdDz0CBZ8idI,"{'compound_code': '9QGP+9Q Portola Valley, Cal...",4.8,ChIJbQOGXoevj4ARdDz0CBZ8idI,"[secondary_school, school, point_of_interest, ...",17,True
96,OPERATIONAL,"3750 Voigt Dr, La Jolla, CA 92037, United States","{'location': {'lat': 32.882748, 'lng': -117.22...",https://maps.gstatic.com/mapfiles/place_api/ic...,4b51bf9efb74bc9e768699cd0b21c8738188f736,Preuss School UCSD,NaN,"[{'height': 1872, 'html_attributions': ['<a hr...",ChIJ-cZwHscG3IARZRKePqjA-SE,"{'compound_code': 'VQMH+34 La Jolla, San Diego...",4.8,ChIJ-cZwHscG3IARZRKePqjA-SE,"[secondary_school, school, point_of_interest, ...",29,NaN
97,OPERATIONAL,"1175 W Baseline Rd, Claremont, CA 91711, Unite...","{'location': {'lat': 34.1233117, 'lng': -117.7...",https://maps.gstatic.com/mapfiles/place_api/ic...,411ce08b0fa7daad33c5f2d00005b29105b96c86,The Webb Schools,{'open_now': False},"[{'height': 3687, 'html_attributions': ['<a hr...",ChIJGYV4Ftgvw4ARiFXgV5GVsow,"{'compound_code': '47F6+8F Claremont, Californ...",4.4,ChIJGYV4Ftgvw4ARiFXgV5GVsow,"[secondary_school, school, point_of_interest, ..."

In [31]:
bars = []
for b in barset["geometry"]:
    new_dict={}
    new_dict["Latitude"]= st['location']['lat']
    new_dict["Longitude"]= st['location']['lng']
    bars.append(new_dict)

In [32]:
bars = pd.DataFrame(bars)
bars

,Latitude,Longitude
0,35.36891,-119.018524
1,35.36891,-119.018524
2,35.36891,-119.018524
3,35.36891,-119.018524
4,35.36891,-119.018524
...,...,...
95,35.36891,-119.018524
96,35.36891,-119.018524
97,35.36891,-119.018524
98,35.36891,-119.018524


In [33]:
FindDistances(bars)

In [34]:
offices = offices.rename(columns={"New":"Bars"})

In [35]:
offices

,Name,Latitude,Longitude,State Code,Starbucks,Bars
0,Mahalo,34.017606,-118.487267,CA,0.374018,1.451984
1,Ustream,37.392936,-122.079480,CA,0.072979,3.669623
2,SodaHead,37.269175,-119.306607,CA,1.717303,1.921977
3,Funny Or Die,37.421621,-122.137444,CA,0.008468,3.733803
4,Seesmic,37.775196,-122.419204,CA,0.023097,4.165913
...,...,...,...,...,...,...
138,Global Fitness Media,34.054318,-118.462876,CA,0.340339,1.427199
139,CrowdGather,34.168955,-118.575293,CA,0.444026,1.279198
140,High Gear Media,37.445335,-122.158330,CA,0.023320,3.764295
141,Teambox,41.406335,2.192023,CA,119.644106,121.360814


## Airports

**Requisite:**  
Account managers need to travel a lot.

In [36]:
data = getLocations("Airport in California", 100)
airportset = pd.DataFrame(data)

In [37]:
airports = []
for a in airportset["geometry"]:
    new_dict={}
    new_dict["Latitude"]= a['location']['lat']
    new_dict["Longitude"]= a['location']['lng']
    airports.append(new_dict)

In [38]:
airports = pd.DataFrame(airports)
airports

,Latitude,Longitude
0,33.941589,-118.408530
1,37.621313,-122.378955
2,33.676190,-117.867476
3,34.055978,-117.598057
4,38.695085,-121.590065
...,...,...
95,34.259805,-118.411929
96,38.659418,-121.395548
97,36.934261,-121.787818
98,34.084844,-118.033012


In [39]:
FindDistances(airports)

In [40]:
offices = offices.rename(columns={"New":"Airports"})

In [41]:
offices

,Name,Latitude,Longitude,State Code,Starbucks,Bars,Airports
0,Mahalo,34.017606,-118.487267,CA,0.374018,1.451984,0.109445
1,Ustream,37.392936,-122.079480,CA,0.072979,3.669623,0.153308
2,SodaHead,37.269175,-119.306607,CA,1.717303,1.921977,0.642420
3,Funny Or Die,37.421621,-122.137444,CA,0.008468,3.733803,0.216336
4,Seesmic,37.775196,-122.419204,CA,0.023097,4.165913,0.159060
...,...,...,...,...,...,...,...
138,Global Fitness Media,34.054318,-118.462876,CA,0.340339,1.427199,0.125145
139,CrowdGather,34.168955,-118.575293,CA,0.444026,1.279198,0.094820
140,High Gear Media,37.445335,-122.158330,CA,0.023320,3.764295,0.243403
141,Teambox,41.406335,2.192023,CA,119.644106,121.360814,119.661965


In [42]:
offices.sort_values(by=["Airports", "Starbucks", "Bars"])

,Name,Latitude,Longitude,State Code,Starbucks,Bars,Airports
111,StockTwits,32.716190,-117.168697,CA,1.144072,3.234005,0.030260
127,Smallaa,37.394580,-121.928393,CA,0.057502,3.545515,0.030638
35,RocketOn,37.654660,-122.407880,CA,0.138201,4.088078,0.044144
75,My Damn Channel,33.901954,-118.387202,CA,0.345928,1.597037,0.045009
132,textPlus,33.984469,-118.436790,CA,0.339952,1.501697,0.051355
...,...,...,...,...,...,...,...
119,Socialize,38.885210,-76.998641,CA,40.516672,42.166752,40.531473
141,Teambox,41.406335,2.192023,CA,119.644106,121.360814,119.661965
47,XIHA,47.361229,8.551597,CA,126.479763,128.132555,126.495298
81,Zendesk,55.676294,12.568116,CA,131.583274,133.144409,131.595603


## Average of distances